<a href="https://colab.research.google.com/github/gitmayanksingh/Natural-Language-Processing/blob/master/GPT2_text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 221, done.
remote: Total 221 (delta 0), reused 0 (delta 0), pack-reused 221
Receiving objects: 100% (221/221), 4.37 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [0]:
import os

In [2]:
pwd

'/content'

In [0]:
os.chdir("gpt-2")

In [4]:
pwd

'/content/gpt-2'

In [6]:
#pip install tensorflow-gpu
pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 40kB/s 
     |████████████████████████████████| 3.8MB 43.7MB/s 
     |████████████████████████████████| 512kB 51.9MB/s 
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorflow-gpu 1.14.0
    Uninstalling tensorflow-gpu-1.14.0:
      Successfully uninstalled tensorflow-gpu-1.14.0


In [7]:
pip install -r requirements.txt

In [5]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.23Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 64.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.11Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:18, 74.8Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 10.9Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 64.2Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 62.2Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')

In [10]:
import json
import os
import numpy as np
import tensorflow as tf

In [0]:
import model, sample, encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [13]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> I went to my friend's birthday and there I saw 
======================================== SAMPLE 1 ========================================
ʹʃʒ laughing their æˊʒ augmented aura full of tangent angles ¹ʒ, as if just being able to see his momentary blurry fingers on my breast enticed me ... a broken flush was approaching. ʃʒ, ʒ, that turned me on enough that even those small panics that really aren´t ʒ so bad weren´t going away. ʃʒ He was pointing toward me, but it made ʒ stuff fine, I felt like going back there where ʒ almost was... ʒ Was ʒ this way ʒ, but that´s because of his customisation. ʒ That *ALF never smiled, because he never took as much photos as Karen extends in the major

KeyboardInterrupt: ignored